# Conexão com drive

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Atividade 1: Remoção de comentários do BTGPactual

Este processo é essencial pois não queremos uma seleção enviada no pré processamento, e entendemos que os comentários do Instagram do BTG Pactual não irá comentar de forma negativa nenhuma de duas postagens.

### 1.1.1. Setup

In [28]:
import pandas as pd
from textblob import TextBlob

Pandas: é uma biblioteca de análise de dados em Python que oferece estruturas de dados e ferramentas para manipulação e análise de dados. Com o Pandas, é possível ler, escrever e manipular dados em diversos formatos, como CSV, Excel, SQL, etc. Ele oferece uma grande quantidade de funcionalidades para trabalhar com dados em Python, incluindo a capacidade de filtrar, agregar, reorganizar e transformar dados de várias maneiras.

TextBlob: é uma biblioteca de processamento de linguagem natural em Python. Ela oferece uma interface simples para tarefas comuns de NLP, como análise de sentimentos, correção ortográfica, extração de frases-chave e classificação de texto. O TextBlob é construído sobre a biblioteca NLTK (Natural Language Toolkit) e oferece uma sintaxe fácil de usar para muitas tarefas de processamento de linguagem natural. É uma biblioteca muito útil para análise de texto em Python.

### 1.1.2. Lendo CSV

In [29]:
dados = pd.read_csv('/content/drive/MyDrive/PROJETO BTG/Base Inicial.csv', delimiter=',', encoding='ISO-8859-1')

### 1.1.2. Teste Isolado

### 1.1.3. Definição de Função

Função de remoção de autor

In [30]:
def filtrar_dados_por_autor(dados, autor):
    # Filtra os dados por autor, removendo as linhas em que a coluna 'autor' contém a string passada como parâmetro.
    # Substitui os valores nulos (NaN) da coluna 'autor' por 'desconhecido'.
    # Retorna um novo DataFrame com as linhas filtradas.
    # Substitui os valores nulos (NaN) da coluna 'autor' por 'desconhecido'.
    dados['autor'] = dados['autor'].fillna('desconhecido')

    # Cria um novo DataFrame chamado 'dados_filtrados', que contém todas as linhas do DataFrame original (dados), exceto aquelas em que a coluna 'autor' contém a string passada como parâmetro.
    # O operador ~ é usado para selecionar todas as linhas em que a condição 'dados['autor'].str.contains(autor)' é falsa.
    dados_filtrados = dados[~dados['autor'].str.contains(autor)]
    
    return dados_filtrados

#### 1.1.3.1. Testes da Função de Remoção de autor

In [31]:
dados_filtrados = filtrar_dados_por_autor(dados, "autor")
print(dados_filtrados)

      Column1       id           autor  \
0           0        1      btgpactual   
1           1        2      btgpactual   
2           2        3      btgpactual   
3           3        4      btgpactual   
4           4        5      btgpactual   
...       ...      ...             ...   
4544     4544  3028308   caioaugustofr   
4545     4545  3028501      btgpactual   
4546     4546  3028580      btgpactual   
4547     4547  3029097  je_mascarenhas   
4548     4548  3029149       linequer_   

                                                  texto    shortCode  \
0     OlÃ¡, Ana Paula! Como vai?\n\nSentimos muito p...  CkG9KXjpY2B   
1     OlÃ¡, Rosana! Tudo bem?\n\nPara te orientarmos...  CkG9KXjpY2B   
2                    @luizantoniojovinodossan ð ð  CkG9KXjpY2B   
3     @ronialencar Boa tarde! Tudo bem?\n\nLamentamo...  CkG9KXjpY2B   
4     Oi, Nelson! Tudo bem?\n\nPossuÃ­mos soluÃ§Ãµes...  CkG9KXjpY2B   
...                                                 ...        

### 1.1.4. Tratamento de dados nulos

In [32]:
# identifica células com valores nulos
dados.isnull()

# conta o número de valores nulos em cada coluna
dados.isnull().sum()

Column1                    0
id                         0
autor                      0
texto                    100
shortCode                128
tipoInteracao            128
tipoMidia                128
idInteracao              128
idPai                    128
URL                      128
dataPublicada            128
dataInsercao             128
anomalia                 128
probabilidadeAnomalia    128
processado               156
contemHyperlink          156
dtype: int64

Após essa operação necessitamos ver na coluna texto - onde estão os comentários - as linhas vazias para que seja substituída por uma string vazia para que possamos analisar o sentimento das palavras.

In [33]:
# conta o número total de valores nulos na coluna texto do DataFrame
dados['texto'].isnull().sum().sum()

100

Nesse código, está sendo preenchido com uma string vazia os valores nulos (NaN) da coluna 'texto' do DataFrame 'dados_filtrados'. Isso é feito com o método 'fillna' do pandas, que substitui os valores nulos pelo valor especificado como argumento, no caso, uma string vazia ('').

O resultado da operação é atribuído à variável 'dados_depois', mas também é atribuído ao próprio DataFrame 'dados_filtrados' (de forma redundante), com a finalidade de atualizar o valor no DataFrame original.

In [34]:
# preenche os valores vazios na coluna 'texto' com uma string vazia
dados_filtrados['texto'] = dados_filtrados['texto'].fillna('')

<ipython-input-34-a7d2b333df5f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_filtrados['texto'] = dados_filtrados['texto'].fillna('')


# Atividade 2: Criando função que analisa o sentimento do texto.
A função utiliza a biblioteca TextBlob para analisar a polaridade do sentimento do texto. Se a polaridade for maior que 0, é considerado como sentimento positivo. Se for menor que 0, é considerado como sentimento negativo. Caso contrário, é considerado como neutro.

In [35]:
# define a função 'analisar_sentimento' que recebe um texto e retorna um sentimento (positivo, negativo ou neutro)
def analisar_sentimento(texto):
    # cria um objeto TextBlob a partir do texto
    tb = TextBlob(texto)
    # extrai a polaridade do sentimento do objeto TextBlob
    polaridade = tb.sentiment.polarity
    # compara a polaridade para determinar o sentimento (positivo, negativo ou neutro)
    if polaridade > 0:
        return 'positivo'
    elif polaridade < 0:
        return 'negativo'
    else:
        return 'neutro'

# aplica a função 'analisar_sentimento' à coluna 'texto' e armazena o resultado na coluna 'sentimento'
dados_filtrados['sentimento'] = dados_filtrados['texto'].apply(analisar_sentimento)

# conta a quantidade de cada tipo de sentimento na coluna 'sentimento'
sentimentos = dados_filtrados['sentimento'].value_counts()

# exibe a quantidade de cada tipo de sentimento
print(sentimentos)

neutro      4253
positivo     212
negativo      83
Name: sentimento, dtype: int64


<ipython-input-35-425651b4a20c>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_filtrados['sentimento'] = dados_filtrados['texto'].apply(analisar_sentimento)


# Atividade 3: Teste